In [1]:
# globals (dev)
FOLDER_MODULES = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules'  
FOLDER_SHARED = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared'
GRP_LYR_FILE = r"C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\lyr\group_template.lyrx"

# set gdal global environ
import os
os.environ['GDAL_DISABLE_READDIR_ON_OPEN'] = 'EMPTY_DIR'
os.environ['CPL_VSIL_CURL_ALLOWED_EXTENSIONS '] = 'tif'
os.environ['VSI_CACHE '] = 'TRUE'
os.environ['GDAL_HTTP_MULTIRANGE '] = 'YES'
os.environ['GDAL_HTTP_MERGE_CONSECUTIVE_RANGES '] = 'YES'

# also set rasterio env variables
rasterio_env = {
    'GDAL_DISABLE_READDIR_ON_OPEN': 'EMPTY_DIR',
    'CPL_VSIL_CURL_ALLOWED_EXTENSIONS': 'tif',
    'VSI_CACHE': True,
    'GDAL_HTTP_MULTIRANGE': 'YES',
    'GDAL_HTTP_MERGE_CONSECUTIVE_RANGES': 'YES'
}

# disable future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# safe imports
import sys                  # arcgis comes with these
import datetime                 # arcgis comes with these
import numpy as np              # arcgis comes with these
import arcpy                    # arcgis comes with these
from datetime import datetime   # arcgis comes with these

# risky imports (not native to arcgis)
try:
    from osgeo import gdal
    from osgeo import ogr
    import tempfile
    import xarray as xr
    import dask
    import rasterio
    import pystac_client
    from odc import stac
except:
    arcpy.AddError('Python libraries xarray, dask, rasterio, pystac, or odc not installed.')
    raise

# import tools
try:
    # shared folder
    sys.path.append(FOLDER_SHARED)
    import arc, satfetcher, tools

    # module folder
    sys.path.append(FOLDER_MODULES)
    import nrt, cog_odc, cog
except:
    arcpy.AddError('Could not find tenement tools python scripts (modules, shared).')
    raise

In [2]:
import os
import shutil
import datetime
import arcpy

tbx = r"C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\toolbox\tenement-tools-toolbox.pyt"
arcpy.ImportToolbox(tbx)

<module 'toolbox'>

In [3]:
import matplotlib.pyplot as plt

In [4]:
# need button to create shapefile with relevent attributes
# set a folder
# create a shapefile of polygons
# need a id field, out netcdf, platform, start/end monitor date, other

### Create monitoring areas

In [4]:
out_folder = r'C:\Users\Lewis\Desktop\nrt_projects'
out_filename = 'ophthalmia_monitoring'

# create a new nrt project
arcpy.NRT_Create_Project_toolbox(out_folder, out_filename)

Creating new monitoring project database...


Traceback (most recent call last):
  File "<string>", line 7225, in execute
  File "C:\Users\Lewis\Documents\GitHub\tenement-tools\modules\nrt.py", line 65, in create_nrt_project
    raise ValueError('Requested file location arleady exists. Choose a different name.')
ValueError: Requested file location arleady exists. Choose a different name.


ExecuteError: ERROR 000582: Error occurred during execution.


### Make monitoring areas

In [ ]:
# this is up to user using usual arcgis pro tools?

### Iterate monitoring areas and update cubes

In [ ]:
# perform sync
in_feat = r'C:\Users\Lewis\Desktop\nrt_projects\ophthalmia_monitoring.gdb\monitoring_areas'

# create a new nrt project
arcpy.NRT_Sync_Cube_toolbox(in_feat=in_feat)

### Do thing

In [ ]:
# temp inputs
in_fmask_flags = 'Valid;Snow;Water'
in_max_cloud = 0
in_veg_idx = 'MAVI'

In [5]:
# THIS IS FOR NRT_Detect_Change
def execute(in_feat, in_fmask_flags, in_max_cloud=0, in_veg_idx='MAVI'):
    """
    """
        
    # grab parameter values 
    #in_feat = parameters[0]        # input monitoring area features
    
    
    # # # # #
    # notify user and set up progress bar
    arcpy.AddMessage('Beginning NRT change detection...')
    arcpy.SetProgressor(type='step', 
                        message='Preparing parameters...', 
                        min_range=0, max_range=3)
            
    # prepare features shapefile
    shp_desc = arcpy.Describe(in_feat)
    in_feat = os.path.join(shp_desc.path, shp_desc.name)
    
    # validate monitoring area feature class
    if not nrt.validate_monitoring_areas(in_feat):
        arcpy.AddError('Monitoring areas feature is invalid.')
        raise
    
    # get input featureclass file, get dir and filename
    in_name = os.path.basename(in_feat)     # name of monitor fc
    in_gdb = os.path.dirname(in_feat)       # path of gdb

    # check gdv extension
    if not in_gdb.endswith('.gdb'):
        arcpy.AddError('Feature class is not in a geodatabase.')
        raise
    else:
        in_path = os.path.splitext(in_gdb)[0]   # path of gdb without ext
        in_data_path = in_path + '_' + 'cubes'  # associated cube data folder

    # check if cubes folder exists
    if not os.path.exists(in_data_path):
        arcpy.AddError('Could not find cube folder for selected monitoring areas.')
        raise    
        
    # convert fmask flags as text to numeric code equivalents
    in_fmask_flags = [e for e in in_fmask_flags.split(';')]
    in_fmask_flags = arc.convert_fmask_codes(in_fmask_flags)
    
    # check if cloud cover is valid
    if in_max_cloud < 0 or in_max_cloud > 100:
        arcpyAddError('Cloud cover must be between 0 and 100.')
        raise
        
        
    # todo count num feats in fc for progressor 
    #


    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Iterating through monitoring areas...')
    arcpy.SetProgressorPosition(1)
        
    # get features (will always have at least one, as we validated earlier)
    driver = ogr.GetDriverByName("OpenFileGDB")
    data_source = driver.Open(os.path.dirname(in_feat), 0)
    feats = data_source.GetLayer('monitoring_areas')

    # iterate through each polygon
    for feat in feats:
        
        # # # # #
        # notify
        arcpy.AddMessage('Validating monitoring area: {}'.format(feat['area_id']))

        # send off to check if valid
        is_valid = nrt.validate_monitoring_area(area_id=feat['area_id'],
                                                platform=feat['platform'], 
                                                s_year=feat['s_year'], 
                                                e_year=feat['e_year'], 
                                                index=feat['index'])
                
        # check if monitoring area is valid
        if not is_valid:
            arcpy.AddWarning('Invalid monitoring area: {}, skipping.'.format(feat['area_id']))
            continue
        
        
        # # # # #
        # notify
        arcpy.AddMessage('Opening cube for monitoring area: {}'.format(feat['area_id']))

        # set existing nc path and open if exists
        out_nc = os.path.join(in_data_path, 'cube' + '_' + feat['area_id'] + '.nc')
        if os.path.exists(out_nc):
            try:
                ds_existing = xr.open_dataset(out_nc)                   
            except:
                arcpy.AddWarning('Could not open existing cube, skipping.')
                continue
        
        
        # # # # #
        # notify
        arcpy.AddMessage('Preparing cube for monitoring area: {}'.format(feat['area_id']))
        
        # check xr type, dims, num time
        if not isinstance(ds_existing, xr.Dataset):
            arcpy.AddWarning('NetCDF is not not an xarray dataset type, skipping.')
            continue
        elif 'x' not in list(ds_existing.dims) or 'y' not in list(ds_existing.dims):
            arcpy.AddWarning('No x or y dimension in dataset, skipping.')
            continue
        elif 'time' not in list(ds_existing.dims):
            arcpy.AddWarning('No time dimension in dataset, skipping.')
            continue
        
        # get dataset and band attributes
        ds_attrs = ds_existing.attrs
        ds_band_attrs = ds_existing[list(ds_existing.data_vars)[0]].attrs
        ds_spatial_ref_attrs = ds_existing['spatial_ref'].attrs    
        
        
        # # # # #
        # notify
        #arcpy.SetProgressorLabel('Removing invalid pixels and empty dates...')
        #arcpy.SetProgressorPosition(3)

        # check if expected band name exists
        mask_band = arc.get_name_of_mask_band(list(ds_existing.data_vars))

        # remove invalid pixels and empty scenes
        ds_existing = cog.remove_fmask_dates(ds=ds_existing, 
                                             valid_class=in_fmask_flags, 
                                             max_invalid=in_max_cloud, 
                                             mask_band=mask_band, 
                                             nodata_value=np.nan, 
                                             drop_fmask=True)
        
        
        # # # # #
        # notify and increment progess bar
        #arcpy.SetProgressorLabel('Calculating vegetation index...')
        #arcpy.SetProgressorPosition(4)

        # conform dea aws band names based on platform
        ds_existing = satfetcher.conform_dea_ard_band_names(ds=ds_existing, 
                                                            platform=feat['platform'].lower()) 

        # calculate vegetation index 
        ds_existing = tools.calculate_indices(ds=ds_existing, 
                                              index=in_veg_idx.lower(), 
                                              custom_name='veg_idx', 
                                              rescale=False, 
                                              drop=True)
        
        # append original attributes on to new band
        ds_existing['veg_idx'].attrs = ds_band_attrs
        
        # see phenolopy for resample, edge effects, etc
        # todo!
        
        
        # # # # #
        # notify and increment progess bar
        #arcpy.SetProgressorLabel('Masking out edge pixels...')
        #arcpy.SetProgressorPosition(4)

        # convert feature to layer and use to mask
        geom = ogr.Open(feat.ExportToJson(), 0)
        lyr = geom.GetLayer()
        mask = nrt.mask_xr_via_polygon(geom=lyr, 
                                       x=ds_existing['x'].data, 
                                       y=ds_existing['y'].data, 
                                       bbox=ds_existing.geobox.extent.boundingbox, 
                                       transform=ds_existing.geobox.transform, 
                                       ncols=len(ds_existing['x']), 
                                       nrows=len(ds_existing['y']), 
                                       mask_value=1)
        
        # apply mask to current dataset, set everything outside to nan
        ds_existing = ds_existing.where(mask)
                
        
        # # # # #
        # notify and increment progess bar
        #arcpy.SetProgressorLabel('Preparing cubve for change detection...')
        #arcpy.SetProgressorPosition(4)      
        
        # prepare time into ewmacd compaitble structure
        #time_array = ds_existing['time'].data
        # may need to convert

        # get raw index values
        #veg_array = ds_existing['veg_idx'].median(['x', 'y']).data
        
        # sumamrise each image to 1 median
        ds_summary = ds.median(['x', 'y'])

 
        # # # # #
        # notify and increment progess bar
        #arcpy.SetProgressorLabel('Performing NRT change detection...')
        #arcpy.SetProgressorPosition(4)   
        
        # apply ewmacd function
        #ds_change = nrt.EWMACD(ds=ds, trainingPeriod='dynamic', etc..)
        
 
        # # # # #
        # notify and increment progess bar
        #arcpy.SetProgressorLabel('Building output change detection cube...')
        #arcpy.SetProgressorPosition(4)   
        
        # broadcast median cube onto pixel cubes to steal median cube values
        ds_summary, _ = xr.broadcast(ds_summary, ds)   # want same median value for every pixel per image
        ds_change, _ = xr.broadcast(ds_change , ds)   # want same change value for every pixel per image

        # ensure dimensions in original order
        ds_summary = ds_summary.transpose('time', 'y', 'x')
        ds_change = ds_change.transpose('time', 'y', 'x')
        
        # re mask summary and change cubes as nan pixels now set to non-nan
        ds_summary = ds_summary.where(mask)
        ds_change = ds_change.where(mask)
        

In [6]:
in_fmask_flags = 'Valid;Snow;Water'

ds_existing = execute(in_feat=r'C:\Users\Lewis\Desktop\nrt_projects\ophthalmia_monitoring.gdb\monitoring_areas',
                        in_fmask_flags=in_fmask_flags) # make sure flags are 1, 5, 7 at this point

Removing dates where too many invalid pixels.
Filling invalid pixels with requested nodata value.
Dropping mask band.
Removed invalid images successfully.
Conforming DEA ARD satellite band names.
Satellite band names conformed successfully.
Calculating indices: mavi.
Calculating index: mavi
Renamed default indices.
Calculated indices successfully.


In [35]:
from smtpd import SMTPServer
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

msg = MIMEMultipart()
msg['From'] = 'mrlewie@outlook.com'
msg['To'] = 'mrlewie@outlook.com'
msg['Subject'] = 'New area detected'
message = 'Yep, I found a new area'
msg.attach(MIMEText(message))

server = smtplib.SMTP('smtp.office365.com', 587) # init server
server.ehlo()  # say hi to server
server.starttls()  # secure email with tls encryption
server.ehlo()  # say hi to server again
server.login('mrlewie@outlook.com', 'halfLife1985micr')  # login

server.sendmail('mrlewie@outlook.com', 'mrlewie@outlook.com', msg.as_string())

server.quit()

(221, b'2.0.0 Service closing transmission channel')

In [14]:
arcpy.server.SendEmailWithZipFileAttachment(To='mrlewie@outlook.com',
                                            From='mrlewie@outlook.com',
                                            Subject='Heya!',
                                            Text='Hello, this is a test my dude!',
                                            Zip_File=r"C:\Users\Lewis\Desktop\hello.zip",
                                            Max_File_Size__MB_=10,
                                            SMTP_Email_Server='smtp.office365.com, 587'
                                            )

ExecuteError: PYTHON ERRORS:
Traceback Info:
  File "c:\program files\arcgis\pro\Resources\ArcToolbox\scripts\SendEmailWithZip.py", line 82, in <module>
    send_mail(fromaddr, sendto, subject, text, zipfile, smtpMailServer, smtpUser, smtpPwd)

Error Info:
    <class 'Exception'>: SendEmailError:PYTHON ERRORS:
Traceback Info:
  File "c:\program files\arcgis\pro\Resources\ArcToolbox\scripts\SendEmailWithZip.py", line 49, in send_mail
    smtp = smtplib.SMTP(server)

Error Info:
    <class 'socket.gaierror'>: [Errno 11001] getaddrinfo failed


ERROR, Unable to send email
Failed to execute (SendEmailWithZipFileAttachment).


In [ ]:
# prepare time into ewmacd compaitble structure
time_array = ds_existing['time'].data
# may need to convert

# get raw index values
veg_array = ds_existing['veg_idx'].median(['x', 'y']).data

### EWMACD

In [ ]:
# call 

In [6]:
ds = xr.open_dataset(r"C:\Users\Lewis\Desktop\nrt_projects\ophthalmia_monitoring_cubes\cube_A001.nc")
ds = ds.compute()

#ds.close()

In [16]:
ds['nbart_red'].isel(time=-50).plot()
plt.show()

In [34]:
# convert tif from vectors
arcpy.conversion.FeatureToRaster(in_features=r"C:\Users\Lewis\Desktop\nrt_projects\ophthalmia_monitoring.gdb\monitoring_areas", 
                                 field='area_id', 
                                 out_raster=r"C:\Users\Lewis\Desktop\testing\area.tif", 
                                 cell_size=30)

<Result 'C:\\Users\\Lewis\\Desktop\\testing\\area.tif'>

In [126]:
ds_area = xr.open_dataset(r"C:\Users\Lewis\Desktop\nrt_projects\ophthalmia_monitoring_cubes\cube_A001.nc")

In [127]:
ds_ras = xr.open_rasterio(r"C:\Users\Lewis\Desktop\testing\area.tif")
ds_ras = ds_ras.where(ds_ras == 0, 1)
ds_ras = ds_ras.squeeze(drop=True)

In [128]:
mask = ds_ras.interp_like(ds_area, method='nearest')
mask = mask.where(mask == 1, 0)
ds_area = ds_area.where(mask)

In [129]:
ds_area['nbart_red'].isel(time=0).plot()
plt.show()

In [145]:
ds_medians = ds_area.median(dim=['x', 'y'], keep_attrs=True)
ds_medians['nbart_red'].data

array([1785.5, -999. , 3497. , -999. ,  513. , -999. , 3019. , -999. ,
       -999. ,  680. ,  889.5, -999. ,  457. ,  505. , -999. , -999. ,
        466. , -999. , 2501. , -999. ,  499. ,  538.5, -999. , -999. ,
        572. , -999. ,  655.5, -999. , 3751.5, -999. ,  792. , -999. ,
        844.5, -999. ,  838.5, -999. ,  697. , -999. ,  633. , -999. ,
       2211.5, -999. , 1874.5, -999. , 2520. , -999. ,  490.5, -999. ,
        544.5, -999. ,  516. , -999. ,  503. , -999. ,  497. , -999. ,
        457. , -999. , -999. ,  414.5, -999. ,  461. , -999. , -999. ,
        477.5, -999. ,  550.5, -999. ,  557. , -999. , -999. ,  629. ,
       -999. ,  633. , -999. ,  652.5, -999. , -999. ,  720.5,  740.5,
       -999. ,  637.5, -999. , 3438. , -999. , -999. ,  496. , -999. ,
        521.5, -999. ,  528.5, -999. ,  549. , -999. , 4357. , -999. ,
        741. , -999. ,  551.5, -999. ,  503.5,  544.5, -999. ,  590. ,
       -999. ,  613. , -999. ,  712.5, -999. ,  660.5, -999. ,  767.5,
      

In [ ]:

    
    # check folder exists
    #if not os.path.exists(out_folder):
        #raise ValueError('Requested folder does not exist.')
        
    # check file does not already exist
    #if os.path.exists(out_filepath):
        #raise ValueError('Requested file location arleady exists. Choose a different name.')
    
    # build project geodatbase
    #out_filepath = arcpy.management.CreateFileGDB(out_folder, out_filename)
    
    
    # notify
    print('Generating database feature class...')
    
    # temporarily disable auto-visual of outputs
    arcpy.env.addOutputsToMap = False
    
    # create feature class and wgs84 spatial ref sys
    srs = arcpy.SpatialReference(4326)
    out_feat = arcpy.management.CreateFeatureclass(out_path=out_filepath, 
                                                   out_name='monitoring_areas', 
                                                   geometry_type='POLYGON',
                                                   spatial_reference=srs)
    
    
    # notify
    print('Generating database domains...')
    
    # create platform domain
    arcpy.management.CreateDomain(in_workspace=out_filepath, 
                                  domain_name='dom_platforms', 
                                  domain_description='Platform name (Landsat or Sentinel)',
                                  field_type='TEXT', 
                                  domain_type='CODED')
    
    # generate coded values to platform domain
    dom_values = {'Landsat': 'Landsat', 'Sentinel': 'Sentinel'}
    for dom_value in dom_values:
        arcpy.management.AddCodedValueToDomain(in_workspace=out_filepath, 
                                               domain_name='dom_platforms', 
                                               code=dom_value, 
                                               code_description=dom_values.get(dom_value))


    # notify
    print('Generating database fields...') 
    
    # add area id field to featureclass   
    arcpy.management.AddField(in_table=out_feat, 
                              field_name='area_id', 
                              field_type='TEXT', 
                              field_alias='Area ID',
                              field_length=200,
                              field_is_required='REQUIRED')
            
    
    # notify todo - delete if we dont want defaults
    print('Generating database defaults...')  
    
    # set default platform
    arcpy.management.AssignDefaultToField(in_table=out_feat, 
                                          field_name='platform',
                                          default_value='Landsat')   
           
           
    # notify
    print('Creating NetCDF data folder...') 
    
    # create output folder
    out_nc_folder = os.path.join(out_folder, '{}_cubes'.format(out_filename))
    if os.path.exists(out_nc_folder):
        try:
            shutil.rmtree(out_nc_folder)
        except:
            raise ValueError('Could not delete {}'.format(out_nc_folder))

    # create new folder
    os.makedirs(out_nc_folder)
    
    
    # notify
    print('Adding data to current map...') 
    
    # enable auto-visual of outputs
    arcpy.env.addOutputsToMap = True
    
    try:
        # get active map, add feat
        aprx = arcpy.mp.ArcGISProject('CURRENT')
        mp = aprx.activeMap
        mp.addDataFromPath(out_feat)
    
    except:
        arcpy.AddWarning('Could not find active map. Add monitor areas manually.')        
        
    # notify
    print('Created new monitoring project database successfully.')

In [ ]:
# monitor tool
# select shapefile
# run checks
# loop through each record in shapefile
# get date time of last time
# query stac for all dates above this
# if new records, create new netcdf using odc-stac like func for bb, etc
# append to existing netcdf and save

# get input shapefile file, get dir and filename
out_path = os.path.dirname(out_shp)
out_name = os.path.basename(out_shp)

fields = ['AreaID', 'NetCDF', 'Platform', 'VegIdx', 'YrStart', 'YrEnd', 'Shape@']
with arcpy.da.UpdateCursor(out_shp, fields) as cursor:
    for row in cursor:
        area_id = row[0]
        nc = row[1]
        platform = row[2]
        veg_idx = row[3]
        year_start = row[4]
        year_end = row[5]
        geom = row[6]

        # temp
        in_epsg = 3577
        in_res = 30

        # get as bbox
        bbox = [geom.extent.XMin, geom.extent.YMin, 
                geom.extent.XMax, geom.extent.YMax]
        
        # get collections and bands based on platform
        if platform == 'Landsat':
            collections = ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
            bands = ['nbart_red', 'nbart_green', 'nbart_blue']
        else:
            raise ValueError('Not yet implemented')

        # if netcdf field is empty, add path to row
        if nc == ' ' or nc == '' or nc is None:
            
            # notify
            print('Querying stac for new Area ID: {}'.format(area_id))
            
            # update row with nc path
            out_nc_path = os.path.join(out_path, 'area_{}.nc'.format(area_id))
            row[1] = out_nc_path
            
            # get dates
            in_from_date = '{}-01-01'.format(year_start)
            #in_to_date = '{}-12-31'.format(datetime.now().year)  # testing
            in_to_date = '2020-03-01'
            
            # notify
            print('Getting new data for period: {} to {}'.format(in_from_date, in_to_date))
            
            # get me the data!
            ds = fetch_odc_xr(collections=collections, 
                              in_from_date=in_from_date, 
                              in_to_date=in_to_date, 
                              bbox=bbox, 
                              bands=bands, 
                              in_epsg=3577, 
                              in_res=30, 
                              like=None)
                
            # download and export netcdf to output folder
            with rasterio.Env(**rasterio_env):
                tools.export_xr_as_nc(ds=ds, filename=out_nc_path)
           
        else:
            # notify
            print('Querying stac for existing Area ID: {}'.format(area_id))
            
            # get output nc path
            out_nc_path = nc
            
            # load existing netcdf
            ds_old = xr.open_dataset(out_nc_path)
                  
            # notify
            print('Existing cube has {} images'.format(len(ds_old['time'])))
            
            # get latest datetime from ds old
            latest_dt = ds_old.isel(time=-1)
            in_from_date = str(latest_dt['time'].dt.strftime('%Y-%m-%d').values)                                    
            
            # get now
            in_to_date = '{}-12-31'.format(datetime.now().year)  # testing
            #in_to_date = '2021-06-01'
            
            
                   
            # notify
            print('Adding data for period: {} to {}'.format(in_from_date, in_to_date))
                  
            # get me the data!
            ds_new = fetch_odc_xr(collections=collections, 
                                  in_from_date=in_from_date, 
                                  in_to_date=in_to_date, 
                                  bbox=bbox, 
                                  bands=bands, 
                                  in_epsg=None, 
                                  in_res=None, 
                                  like=ds_old)
                  
            # notify
            print('New cube has {} images'.format(len(ds_new['time'])))
            
            # download and compute
            with rasterio.Env(**rasterio_env):
                ds_new = ds_new.compute()            
            
            # combine but exclude duplicates CHECK THIS CAREFULLY
            ds_old = ds_old.combine_first(ds_new)
                  
            # notify
            print('Newly synced cube now has {} images'.format(len(ds_old['time'])))
                  
            # export new to named file temp, close old, then overwrite
            with tempfile.NamedTemporaryFile() as tmp:
                ds_old.to_netcdf(tmp.name + '.nc')
                
                ds_old.close()
                ds_new.close()
                del ds_old
                del ds_new
                
                ds = xr.open_dataset(tmp.name + '.nc')
                
                ds.to_netcdf(out_nc_path)

            
        # update cursor regardless
        cursor.updateRow(row)
